#### Randy Baicich                                                                                    

# Capstone Project 1: 

# *Shark Attacks in Coastal Waters*

The dataset sourced to be used in this project was originally collected from the Global Shark Attack file on the [Shark Research Institute's website](https://www.sharkattackfile.net). The dataset is available for download from [Kaggle](https://www.kaggle.com/c/shark-attack-dataset).

*In this notebook, I will use the Shark Attack dataset to perform a comprehensive analysis of the data. The first step is to import the necessary libraries and modules to facilitate data processing and analysis. Once the dataset is imported, the next step is to ensure that the data is tidy, which involves organizing and cleaning it for further analysis. This includes tasks such as removing unnecessary columns, capitalizing column names, and removing extra spaces. After tidying the data, the next step is to transform it by performing various calculations to derive meaningful insights. This may involve calculating statistics such as counts or averages as well as creating new features based on existing data. I will also attempt to connect to an [ElephantSQL](https://www.elephantsql.com/) instance, to perform queries using the created tables. Once connected to the instance the transformed data is then used to visualize the data. I will create several visualizations, including scatter plots and bar charts, as well as use [Tableu](https://public.tableau.com/app/discover) to explore relationships between different variables and gain insights into the patterns and distributions present in the data. Finally, the last step is to communicate the findings and insights obtained from the analysis. This includes summarizing the key findings, presenting visualizations, and providing interpretations and recommendations based on the results.*

# Starting Hypothesis.

#### *Proceeding the analysis of the Shark Attack dataset, I hypothesize that a select few shark species will be responsible for the majority of shark attacks and fatalities, rather than a wide variety of species. Additionally, I believe that the majority of shark attacks will occur at specific times, indicating a temporal pattern in shark-human interactions. By exploring the data and conducting statistical analysis, I aim to determine if these hypotheses hold true and gain insights into the key factors influencing shark attacks and fatalities.*

## Part 1: *Import, Clean, and Save Data.*

#### *Import all necessary libraries.*

In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

#### *Import the sourced Shark Attack data.*

In [2]:
df = pd.read_csv(r'C:\Users\RedneckRandy\Documents\GitHub\Capstone-Project-1\GSAF5.csv', low_memory=False)


#### *Clean the CSV file/data.*

In [3]:
#Capitalize all columns
df.columns = [col.capitalize().strip() for col in df.columns]
df

,Index,Case number,Date,Year,Type,Country,Area,Location,Activity,Name,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,0,2020.02.05,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020.01.17,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020.01.16,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020.01.13,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,6457,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6458,6458,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6459,6459,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6460,6460,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Remove extra space in column names
df.columns = df.columns.str.replace(' ', '')
df

,Index,Casenumber,Date,Year,Type,Country,Area,Location,Activity,Name,...,Unnamed:246,Unnamed:247,Unnamed:248,Unnamed:249,Unnamed:250,Unnamed:251,Unnamed:252,Unnamed:253,Unnamed:254,Unnamed:255
0,0,2020.02.05,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020.01.17,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020.01.16,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020.01.13,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,6457,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6458,6458,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6459,6459,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6460,6460,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Drop the first column
df = df.drop(df.columns[0], axis=1)
df

,Casenumber,Date,Year,Type,Country,Area,Location,Activity,Name,Unnamed:9,...,Unnamed:246,Unnamed:247,Unnamed:248,Unnamed:249,Unnamed:250,Unnamed:251,Unnamed:252,Unnamed:253,Unnamed:254,Unnamed:255
0,2020.02.05,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020.01.17,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020.01.16,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020.01.13,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6458,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6459,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6460,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Remove columns starting with "Unnamed"
df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
df

,Casenumber,Date,Year,Type,Country,Area,Location,Activity,Name,Age,...,Fatal(y/n),Time,Species,Investigatororsource,Pdf,Hrefformula,Href,Casenumber.1,Casenumber.2,Originalorder
0,2020.02.05,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,NaN,...,N,09h40,Tiger shark,"K. McMurray, TrackingSharks.com",NaN,NaN,NaN,NaN,NaN,NaN
1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,24,...,N,NaN,NaN,"K. McMurray, TrackingSharks.com",NaN,NaN,NaN,NaN,NaN,NaN
2,2020.01.17,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,59,...,N,08h00,"""A small shark""","B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.17-Schroeter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.17,2020.01.17,6502.0
3,2020.01.16,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,13,...,N,20h30,Broadnose seven gill shark?,"K. McMurray, TrackingSharks.com",2020.01.16-King.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.16,2020.01.16,6501.0
4,2020.01.13,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,26,...,N,14h33,NaN,"C. Creswell, GSAF",2020.01.13-Horne.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.13,2020.01.13,6500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,NaN,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",NaN,NaN,NaN,NaN,NaN,NaN
6458,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,NaN,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",NaN,NaN,NaN,NaN,NaN,NaN
6459,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,NaN,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",NaN,NaN,NaN,NaN,NaN,NaN
6460,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,NaN,...,Y,NaN,NaN,"The Sun, 10/20/1938",NaN,NaN,NaN,NaN,NaN,NaN


#### *Save the new cleaned CSV.*

In [8]:
df.to_csv('sharks_sorted.csv', index=False)

## Part 2: *Analysis of the data*.

In [10]:
#Total Count of 'Y' in 'Fatal (Y/N)'
total_Y_fatal = df['Fatal(y/n)'].str.count('Y').sum()

#Print the total count of 'Y' in 'Fatal(Y/N)'
print("Total Fatal Attacks':", total_Y_fatal)

Total Fatal Attacks': 1400.0


In [11]:
#Total Count of 'N' in 'Fatal (Y/N)'
total_N_fatal = df['Fatal(y/n)'].str.count('N').sum()

#Print the total count of 'Y' in 'Fatal(Y/N)'
print("Total Non-Fatal Attacks':", total_N_fatal)

Total Non-Fatal Attacks': 4652.0


In [13]:
#Convert 'Age' column to numeric using .loc
df.loc[:, 'Age'] = pd.to_numeric(df['Age'], errors='coerce')

#Filter the dataframe for records where 'Fatal(y/n)' is 'Y'
df_Y_fatal = df[df['Fatal(y/n)'] == 'Y']

#Calculate the average 'Age' for the filtered dataframe
average_age_Y_fatal = df_Y_fatal['Age'].mean()

#Print the average age
print("Average Age of Fatal Victims:", average_age_Y_fatal)



Average Age of Fatal Victims: 27.400329489291597


In [14]:
#Total count for each unique value in 'Location' column
location_totals = df['Location'].value_counts()

#Print the total count for each unique value in 'Location' column
print("Total count for each unique value in 'Location' column:")
print(location_totals)

Total count for each unique value in 'Location' column:
Location
New Smyrna Beach, Volusia County                                  181
Daytona Beach, Volusia County                                      34
Cocoa Beach, Brevard County                                        26
Ponce Inlet, Volusia County                                        21
Melbourne Beach, Brevard County                                    19
                                                                 ... 
Buffels Bay (near Knysna)                                           1
Pensacola Beach, Escambia County                                    1
Cintsa East, East London                                            1
11 miles off Dog Island in the Gulf of Mexico, Franklin County      1
Below the English fort, Trincomalee                                 1
Name: count, Length: 4148, dtype: int64


In [15]:
#Total count for each unique value in 'Species' column
species_totals = df['Species'].value_counts()

#Print the total count for each unique value in 'Species' column
print("Total count for each unique value in 'Species' column:")
print(species_totals)

Total count for each unique value in 'Species' column:
Species
White shark                                                                                                                     166
Shark involvement prior to death was not confirmed                                                                              105
Invalid                                                                                                                         102
Shark involvement not confirmed                                                                                                  89
Tiger shark                                                                                                                      81
                                                                                                                               ... 
Questionable incident - shark bite may have precipitated drowning                                                                 1
C. leucas too

In [16]:
#Total count for each unique value in 'Activity' column
activity_totals = df['Activity'].value_counts()

#Print the total count for each unique value in 'Activity' column
print("Total count for each unique value in 'Activity' column:")
print(activity_totals)

Total count for each unique value in 'Activity' column:
Activity
Surfing                                1025
Swimming                                932
Fishing                                 459
Spearfishing                            350
Bathing                                 166
                                       ... 
Feeding mullet to sharks                  1
Sight-seeing                              1
Netting pilchards                         1
Surf-fishing                              1
Wreck of large double sailing canoe       1
Name: count, Length: 1516, dtype: int64


In [17]:
#Total count for each unique value in 'Type' column
type_totals = df['Type'].value_counts()

#Print the total count for each unique value in 'Type' column
print("Total count for each unique value in 'Type' column:")
print(type_totals)

Total count for each unique value in 'Type' column:
Type
Unprovoked             4716
Provoked                593
Invalid                 552
Sea Disaster            239
Watercraft              142
Boat                    109
Boating                  92
Questionable             10
Unconfirmed               1
Unverified                1
Under investigation       1
Boatomg                   1
Name: count, dtype: int64


In [21]:
#Convert 'Time' column to numeric using .loc
df.loc[:, 'Time'] = pd.to_numeric(df['Time'], errors='coerce')

#Filter the dataframe for records where 'Fatal(y/n)' is 'Y'
df_Y_fatal = df[df['Fatal(y/n)'] == 'Y']

#Calculate the average 'Age' for the filtered dataframe
average_time_Y_fatal = df_Y_fatal['Time'].mean()

#Print the average age
print("Average Time of Fatal Attacks:", average_time_Y_fatal)

Average Time of Fatal Attacks: 1500.0


# Part 3: *Connecting to the database as well as Creating and Inserting into tables.*

#### *Connect to ElephantSQL*

In [9]:
conn = psycopg2.connect(dbname='gblqlzwo',
                        user='gblqlzwo',
                        password='UkEdnFRHD1w6hKODlEDEqHMIKujC814K',
                        host='rajje.db.elephantsql.com')
cur = conn.cursor()

#### *Create neccessary tables.*

In [10]:
#Define the columns for the table
columns = ['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Age', 'Injury', 'Fatal(Y/N)', 'Time', 'Species']



In [11]:
create_table_query = '''
    CREATE TABLE shark_data(
        "Date" VARCHAR(100),
        "Year" INT,
        "Type" VARCHAR(100),
        "Country" VARCHAR(100),
        "Area" VARCHAR(100),
        "Location" VARCHAR(100),
        "Activity" VARCHAR(100),
        "Name" VARCHAR(100),
        "Age" VARCHAR(100),
        "Injury" VARCHAR(100),
        "Fatal (Y/N)" VARCHAR(100),
        "Time" VARCHAR(100),
        "Species" VARCHAR(100)
    )
'''
cur.execute(create_table_query)

#### *Insert into the created table.*

In [12]:
for _, row in df[columns].iterrows():
    insert_query = '''
        INSERT INTO shark_data ("Date", "Year", "Type", "Country", "Area", "Location",
                                 "Activity", "Name", "Age", "Injury", "Fatal(Y/N)", "Time", "Species")
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    cur.execute(insert_query, tuple(row))

conn.commit()

KeyError: "['Fatal(Y/N)'] not in index"

# Part 4: *Visualize the data and communicate your results.*

#### *Visualization 1: Species vs Type of Attack.*

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['Species'], data['Type'])
plt.xlabel('Species')
plt.ylabel('Type')
plt.title('Species vs Type')
plt.xticks(rotation=90)
plt.show()

#### *Visualization 2: Type of Attack vs Activity Being Performed.*

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['Type'], data['Activity'])
plt.xlabel('Type')
plt.ylabel('Activity')
plt.title('Type vs Activity')
plt.xticks(rotation=90)
plt.show()

#### *Visualization 3: Total Fatalities.*

In [ ]:
fatal_counts = data['Fatal (Y/N)'].value_counts()
plt.figure(figsize=(6, 6))
plt.bar(['Y', 'N'], fatal_counts)
plt.xlabel('Fatal')
plt.ylabel('Count')
plt.title('Fatal (Y/N) Distribution')
plt.show()

#### *Visualization 4: Fatalities by Species.*

In [ ]:
species_fatal_counts = data.groupby('Species')['Fatal (Y/N)'].value_counts().unstack().fillna(0)
plt.figure(figsize=(10, 6))
species_fatal_counts.plot(kind='bar', stacked=True)
plt.xlabel('Species')
plt.ylabel('Count')
plt.title('Species vs Fatal (Y/N)')
plt.xticks(rotation=90)
plt.legend(title='Fatal (Y/N)')
plt.show()

#### *Close Connections*

In [ ]:
cur.close()
conn.close()

# Part 5: *Findings, Summary, and Conclusion.*